In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, CompositeVideoClip, AudioFileClip
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from moviepy.video.fx.resize import resize

In [2]:
source_video_path = 'GOT.mp4'
output_video_path = 'output_short_video.mp4'
interesting_clips = [(6, 10), (13, 17), (23, 25),(35,43),(52,57)]

In [3]:
video = VideoFileClip(source_video_path)


In [4]:
clips = []
for start, end in interesting_clips:
    clip = video.subclip(start, end)
    clips.append(clip)

Sample of how we can use emotions to find suitable soundtrack to use in our video

In [5]:
# def load_video(video_path, max_frames=20, resize=(64, 64)):
#     video_frames = []
#     cap = cv2.VideoCapture(video_path)
#     try:
#         while True:
#             ret, frame = cap.read()
#             if not ret or len(video_frames) >= max_frames:
#                 break
#             frame = cv2.resize(frame, resize)
#             frame = frame / 255.0  # Normalize pixel values
#             video_frames.append(frame)
#     finally:
#         cap.release()
#     return np.array(video_frames)

# def create_video_analysis_model():
#     model = Sequential()
#     model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(None, 64, 64, 3)))
#     model.add(TimeDistributed(MaxPooling2D((2, 2))))
#     model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
#     model.add(TimeDistributed(MaxPooling2D((2, 2))))
#     model.add(TimeDistributed(Flatten()))
#     model.add(LSTM(64, return_sequences=False))
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# # Example use
# model = create_video_analysis_model()
# aggregate_emotions = []
# for clip in clips:
#     clip_frames = load_video(clip.filename)
#     clip_frames = np.expand_dims(clip_frames, axis=0)
#     predicted_mood = model.predict(clip_frames)
#     aggregate_emotions.append(predicted_mood)


# overall_emotion = np.mean(aggregate_emotions)

#not enough time or training data/architecture to train a custon emotion detection model for videos in 4 days, so using dummy emotion variable:

overall_emotion = 0.3

In [6]:
final_clip = concatenate_videoclips(clips)

In [7]:
###using spotify api to find a suitable song from a playlist using the emotion detected
client_credentials_manager = SpotifyClientCredentials(client_id='c6c3d5a693864bef9c87d00ec208b02e', client_secret='7d8c943381a34d7bab9e95faf0f0c22d')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_id = '7dw7t04acCBwk9B9R5i5l4'
results = sp.playlist_items(playlist_id)
suitable_tracks = []
for item in results['items']:
    track = item['track']
    audio_features = sp.audio_features(track['id'])[0]
    if (overall_emotion > 0.5 and audio_features['energy'] > 0.7) or (overall_emotion <= 0.5 and audio_features['energy'] < 0.7):
        suitable_tracks.append(track['uri'])

# print the suitable tracks
for track_uri in suitable_tracks:
    track_id = track_uri.split(':')[-1]
    track_info = sp.track(track_id)
    print("Track Name:", track_info['name'])
    print("Artist:", track_info['artists'][0]['name'])
    print("Album Name:", track_info['album']['name'])
    print("Release Date:", track_info['album']['release_date'])
    print("Track Duration (ms):", track_info['duration_ms'])
    print("Preview URL:", track_info['preview_url'])
    print()


Track Name: Off The Grid
Artist: Kanye West
Album Name: Donda
Release Date: 2021-08-29
Track Duration (ms): 339249
Preview URL: None

Track Name: Hurricane
Artist: Kanye West
Album Name: Donda
Release Date: 2021-08-29
Track Duration (ms): 243187
Preview URL: None

Track Name: Race My Mind
Artist: Drake
Album Name: Certified Lover Boy
Release Date: 2021-09-03
Track Duration (ms): 269883
Preview URL: None

Track Name: Way 2 Sexy (with Future & Young Thug)
Artist: Drake
Album Name: Certified Lover Boy
Release Date: 2021-09-03
Track Duration (ms): 257604
Preview URL: None

Track Name: Belly, The Weeknd - Die For It (ft. Nas)
Artist: Belly
Album Name: See You Next Wednesday
Release Date: 2021-08-27
Track Duration (ms): 199866
Preview URL: None

Track Name: Sharing Locations (feat. Lil Baby & Lil Durk)
Artist: Meek Mill
Album Name: Sharing Locations (feat. Lil Baby & Lil Durk)
Release Date: 2021-08-27
Track Duration (ms): 161052
Preview URL: https://p.scdn.co/mp3-preview/1fd2b7c307a4fdf5a462

for offline access, right now i have to manually download the song.

In [8]:
music = AudioFileClip("D:\CODE\FOG\OffTheGrid.mp3")
start_time = float(input("Enter the starting timestamp (in seconds) for the music track: "))
music = music.subclip(start_time, start_time + final_clip.duration)
final_clip = final_clip.set_audio(music)
final_clip.write_videofile(output_video_path, codec='libx264')

Moviepy - Building video output_short_video.mp4.
MoviePy - Writing audio in output_short_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_short_video.mp4



Moviepy - Done !
Moviepy - video ready output_short_video.mp4
